# Płytka sieć neuronowa w PyTorch

### Zaimportujmy zależności

In [ ]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
from torchvision import transforms
from torchsummary import summary
import matplotlib.pyplot as plt

### Załadujmy dane i od razu przeskalujmy (funkcja ToTensor konwertuje piksele z [0,255] na [0,1]

In [ ]:
train = MNIST('data', train=True, transform=transforms.ToTensor(), download=True)
test = MNIST('data', train=False, transform=transforms.ToTensor())

### Sprawdźmy jak wyglądają dane

In [ ]:
train.data.shape

In [ ]:
train

In [ ]:
train.data[0]

### Zobaczmy konkretne wartości

In [ ]:
plt.imshow(train.data[0].numpy().squeeze(), cmap='gray_r')

In [ ]:
train.targets[0:100]

In [ ]:
train.targets.shape

In [ ]:
test.data.shape

In [ ]:
test.targets.shape

### Popaczkujmy zbiór danych z wykorzystaniem klasy DataLoader i przejrzyjmy paczki

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True) 
test_loader = torch.utils.data.DataLoader(test, batch_size=128) 

In [ ]:
n_batches = len(train_loader)
n_batches

In [ ]:
n_test_batches = len(test_loader)
n_test_batches

In [ ]:
X_sample, y_sample = iter(train_loader).next()

In [ ]:
X_sample.shape

In [ ]:
y_sample.shape

In [ ]:
y_sample

In [ ]:
X_sample[0]

### Spłaszczamy paczkę 
(niespodzianie służy do tego funkcja .view())

In [ ]:
X_flat_sample = X_sample.view(X_sample.shape[0], -1)

In [ ]:
X_flat_sample.shape

In [ ]:
X_flat_sample[0]

### Czas na zaprojektowanie architektury sieci

PyTorch opiera się na modułach (obiektach klasy `torch.nn.Module`), które są łączone w graf obliczeń. Moduły mogą wykorzystywać parametry (obiekty klasy `torch.nn.Parameter`), dla których mogą być automatycznie liczone gradienty i które mogą podlegać optymalizacji.

**Zadanie 1. Zadeklaruj odpowiedni rozmiar wejścia i wyjścia tak, by warstwa wejściowa przyjęła pojedynczo wszystkie piksele obrazka, a warstwa wyjściowa mogła reprezentować każdą z klas**

In [ ]:
n_input = ...
n_dense = 64
n_out = ...

Tworzymy model, składający się sekwencji warstw: wejściowej warstwy liniowej czyli warstwy implementującej operację $\hat{y} = Wx + b$, (zwróćmy uwagę na jej liczbę wejść i wyjść), ukrytej warstwy gęstej o 64 neuronach z sigmoidalną funkcją aktywacji oraz wyjściowej warstwy liniowej (tu również zwróćmy uwagę na liczbę wejść)

In [ ]:
model = nn.Sequential(
    nn.Linear(n_input, n_dense),
    nn.Sigmoid(),
    nn.Linear(n_dense, n_out)
)

In [ ]:
summary(model, (1, n_input))

### Skonfigurujmy hiperparametry

**Zadanie 2. Zajrzyj do dokumentacji biblioteki torch i zadeklaruj entropię skrośną (CrossEntropy) jako funkcję straty oraz SGD jako algorytm optymalizacji - z parametrami modelu jako parametrami do optymalizacji, oraz ze stałą uczenia równą 0.01**

In [ ]:
cost_fxn = 

In [ ]:
optimizer = 

### Zdefinujmy metrykę trafności prognozy

In [ ]:
def accuracy_pct(pred_y, true_y):
  _, prediction = torch.max(pred_y, 1)
  correct = (prediction == true_y).sum().item()
  return (correct / true_y.shape[0]) * 100.0

### Wreszcie nauczmy sieć 
(zwróćmy zwłaszcza uwagę na to co się dzieje w zagnieżdżonej pętli)

In [ ]:
n_epochs = 20 

print('Training for {} epochs. \n'.format(n_epochs))

for epoch in range(n_epochs):
  
  avg_cost = 0.0
  avg_accuracy = 0.0
  
  for i, (X, y) in enumerate(train_loader): # enumerate() pozwala iterować po całym popaczkowanym zbiorze  
    
    # wykonujemy feed-forward:
    X_flat = X.view(X.shape[0], -1)
    y_hat = model(X_flat)
    cost = cost_fxn(y_hat, y)
    avg_cost += cost / n_batches
    
    # propagacja wsteczna i optymalizacja poprzez algorytm spadku gradientu: 
    optimizer.zero_grad() # ustawiamy gradienty w sieci na zero; .backward() accumulates them in buffers
    cost.backward() #obliczamy i zbieramy gradienty
    optimizer.step() #aktualizujemy wagi z wykorzystaniem zebranych gradientów
    
    # obliczamy wartości wyznaczonej wcześniej metryki:
    accuracy = accuracy_pct(y_hat, y)
    avg_accuracy += accuracy / n_batches
    
    if (i + 1) % 100 == 0:
      print('Step {}'.format(i + 1))
    
  print('Epoch {}/{} complete: Cost: {:.3f}, Accuracy: {:.1f}% \n'
        .format(epoch + 1, n_epochs, avg_cost, avg_accuracy)) 

print('Training complete.')

### Testujemy model

**Zadanie 3. Zaimplementuj w gotowej pętli obliczanie trafności na całym zbiorze testowym z wykorzystaniem funkcji accuracy_pct**

In [ ]:
model.eval()

with torch.no_grad(): # dezyaktywacja klasy autograd która oblicza pochodne i rejestruje graf wszystkich operacji wykonanych na tensorze, w tym przypadku chcemy zaoszczędzić pamięc
  
  avg_test_cost = 0.0
  avg_test_acc = 0.0
  
  for X, y in test_loader:
    
    # dokonujemy prognozy: 
    X_flat = X.view(X.shape[0], -1)
    y_hat = model(X_flat)
    
    # obliczamy koszt: 
    cost = cost_fxn(y_hat, y)
    avg_test_cost += cost / n_test_batches
    
    # tu odpowiedź - obliczamy trafność:
    test_accuracy = 
    avg_test_acc += 

print('Test cost: {:.3f}, Test accuracy: {:.1f}%'.format(avg_test_cost, avg_test_acc))